Preprocess and tokenize the article text from the dataframe of the 4 companies to fix the date format issue.

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
with open('df_4_companies_prep.pickle', 'rb') as read_file:
    df_4_companies_prep=pickle.load(read_file)
df_4_companies_prep.head()

,article_date,article_link-href,article_text,article_title,ticker
0,"Apr 01, 2019",https://www.novartis.com/news/media-releases/n...,Acquisition will give Novartis one clinical an...,Novartis adds clinical and preclinical anti-in...,NVS
1,"Jun 15, 2017",https://www.novartis.com/news/media-releases/n...,Cosentyx® is the first and only IL-17A inhibit...,Novartis' Cosentyx shows sustained improvement...,NVS
2,"Jun 04, 2017",https://www.novartis.com/news/media-releases/n...,Study is longest follow-up to date of a BRAF a...,Novartis landmark study of Tafinlar® + Mekinis...,NVS
3,"Apr 30, 2019",https://www.novartis.com/news/media-releases/s...,Collaboration covers proposed trastuzumab bios...,Sandoz enters agreement for proposed trastuzum...,NVS
4,"Jun 23, 2017",https://www.novartis.com/news/media-releases/n...,83% of patients achieved complete remission (C...,Novartis pivotal CTL019 6-month follow-up data...,NVS


Looks good. Need to make the article_text lowercase, remove punctuation, numbers, nonsense text, then tokenize.

In [3]:
import re
import string
from pprint import pprint
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
#import gensim
#import gensim.corpora as corpora
#from gensim.utils import simple_preprocess
#from gensim.models import CoherenceModel
#import spacy
import matplotlib.pyplot as plt
%matplotlib inline
###need to figure out the gensim problem. It can't find the module

In [4]:
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())


In [5]:
df_4_companies_prep['article_text_lower']=df_4_companies_prep.article_text.map(alphanumeric).map(punc_lower)
df_4_companies_prep.head()

,article_date,article_link-href,article_text,article_title,ticker,article_text_lower
0,"Apr 01, 2019",https://www.novartis.com/news/media-releases/n...,Acquisition will give Novartis one clinical an...,Novartis adds clinical and preclinical anti-in...,NVS,acquisition will give novartis one clinical an...
1,"Jun 15, 2017",https://www.novartis.com/news/media-releases/n...,Cosentyx® is the first and only IL-17A inhibit...,Novartis' Cosentyx shows sustained improvement...,NVS,cosentyx® is the first and only il inhibitor...
2,"Jun 04, 2017",https://www.novartis.com/news/media-releases/n...,Study is longest follow-up to date of a BRAF a...,Novartis landmark study of Tafinlar® + Mekinis...,NVS,study is longest follow up to date of a braf a...
3,"Apr 30, 2019",https://www.novartis.com/news/media-releases/s...,Collaboration covers proposed trastuzumab bios...,Sandoz enters agreement for proposed trastuzum...,NVS,collaboration covers proposed trastuzumab bios...
4,"Jun 23, 2017",https://www.novartis.com/news/media-releases/n...,83% of patients achieved complete remission (C...,Novartis pivotal CTL019 6-month follow-up data...,NVS,of patients achieved complete remission cr...


In [6]:
#I think it worked. 
df_4_companies_prep.tail(10)

,article_date,article_link-href,article_text,article_title,ticker,article_text_lower
1254,"March 16, 2019",http://ir.clovisoncology.com/investors-and-new...,Exploratory analyses of ARIEL3 Rubraca data in...,Clovis Oncology to Highlight Rubraca® (rucapar...,CLVS,exploratory analyses of rubraca data in recu...
1255,"January 24, 2019",http://ir.clovisoncology.com/investors-and-new...,Rubraca\n \n ®\n \n (rucaparib)...,Clovis Oncology Announces European Commission ...,CLVS,rubraca\n \n ®\n \n rucaparib ...
1256,"June 06, 2019",http://ir.clovisoncology.com/investors-and-new...,"Dr. M. James Barrett, Chair since inception, r...",Clovis Oncology Names Ginger L. Graham as New ...,CLVS,dr m james barrett chair since inception r...
1257,"August 22, 2019",http://ir.clovisoncology.com/investors-and-new...,"BOULDER, Colo.--(BUSINESS WIRE)--\nClovis Onco...",Clovis Oncology Announces Exercise by Initial ...,CLVS,boulder colo business wire \nclovis onco...
1258,"May 06, 2019",http://ir.clovisoncology.com/investors-and-new...,"BOULDER, Colo.--(BUSINESS WIRE)--\n Clovi...",Clovis Oncology to Present at the Bank of Amer...,CLVS,boulder colo business wire \n clovi...
1259,"March 19, 2019",http://ir.clovisoncology.com/investors-and-new...,Accepted abstracts highlight data from a phase...,Clovis Oncology Announces Presentations at 201...,CLVS,accepted abstracts highlight data from a phase...
1260,"April 02, 2019",http://ir.clovisoncology.com/investors-and-new...,Interim data from an investigator-initiated st...,Clovis Oncology Announces Interim Results from...,CLVS,interim data from an investigator initiated st...
1261,"January 02, 2019",http://ir.clovisoncology.com/investors-and-new...,"BOULDER, Colo.--(BUSINESS WIRE)--\n Clovi...",Clovis Oncology to Present at the 37th Annual ...,CLVS,boulder colo business wire \n clovi...
1262,"December 16, 2019",http://ir.clovisoncology.com/investors-and-new...,Presented at the International Symposium on Th...,Initial Clinical Experience Reported from FAP-...,CLVS,presented at the international symposium on th...
1263,"February 19, 2019",http://ir.clovisoncology.com/investors-and-new...,-- Combinations of ALKS 4230 with Rucaparib an...,Alkermes and Clovis Oncology Initiate Research...,CLVS,combinations of alks with rucaparib and l...


In [8]:
#df_4_companies_prep.at[1, 'article_text_lower']

Numbers are gone and everything is lowercase. There are still quotation marks, and nonsense characters. Also I'm wondering if I should remove the "about" and "forward-looking statements" from each company's press releases. That might reduce the noise.
Maybe I could do TF-IDF within each company to reduce the noise from the repeated about and forward-looking statements.

Can I take out "this press release includes forward looking statements " and everything after?

In [8]:
!pip install textblob

In [9]:
import sys
sys.path.append("/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages")

In [10]:
from textblob import TextBlob

In [11]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
stop=stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amysillman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/amysillman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
#remove nonsense text. This is from stackoverflow 
#https://stackoverflow.com/questions/52787309/removing-nonsense-words-in-python

words = set(nltk.corpus.words.words())

def clean_sent(sent):
    return " ".join(w for w in nltk.wordpunct_tokenize(sent) \
     if w.lower() in words or not w.isalpha())

df_4_companies_prep['article_text_clean'] = df_4_companies_prep['article_text_lower'].apply(clean_sent)

In [14]:
df_4_companies_prep.head(10)

,article_date,article_link-href,article_text,article_title,ticker,article_text_lower,article_text_clean
0,"Apr 01, 2019",https://www.novartis.com/news/media-releases/n...,Acquisition will give Novartis one clinical an...,Novartis adds clinical and preclinical anti-in...,NVS,acquisition will give novartis one clinical an...,acquisition will give one clinical and two pre...
1,"Jun 15, 2017",https://www.novartis.com/news/media-releases/n...,Cosentyx® is the first and only IL-17A inhibit...,Novartis' Cosentyx shows sustained improvement...,NVS,cosentyx® is the first and only il inhibitor...,® is the first and only inhibitor to show sust...
2,"Jun 04, 2017",https://www.novartis.com/news/media-releases/n...,Study is longest follow-up to date of a BRAF a...,Novartis landmark study of Tafinlar® + Mekinis...,NVS,study is longest follow up to date of a braf a...,study is follow up to date of a and inhibitor ...
3,"Apr 30, 2019",https://www.novartis.com/news/media-releases/s...,Collaboration covers proposed trastuzumab bios...,Sandoz enters agreement for proposed trastuzum...,NVS,collaboration covers proposed trastuzumab bios...,collaboration in phase development for human e...
4,"Jun 23, 2017",https://www.novartis.com/news/media-releases/n...,83% of patients achieved complete remission (C...,Novartis pivotal CTL019 6-month follow-up data...,NVS,of patients achieved complete remission cr...,of complete remission or with incomplete blood...
5,"May 25, 2016",https://www.novartis.com/news/media-releases/n...,Outlines actions underway to accelerate launch...,Novartis highlights its strong foundation for ...,NVS,outlines actions underway to accelerate launch...,underway to accelerate launch of ® further exp...
6,"Nov 05, 2018",https://www.novartis.com/news/media-releases/n...,26 potential blockbusters in confirmatory deve...,Novartis R&D update highlights industry leadin...,NVS,potential blockbusters in confirmatory devel...,potential in confirmatory development in clini...
7,"May 01, 2018",https://www.novartis.com/news/media-releases/k...,Kymriah demonstrated an overall response rate ...,"Kymriah® (tisagenlecleucel), first-in-class CA...",NVS,kymriah demonstrated an overall response rate ...,an overall response rate of with median durati...
8,"Jun 28, 2016",https://www.novartis.com/news/media-releases/n...,Agreement is the latest in a series of acquisi...,Novartis adds bispecific antibodies to its gro...,NVS,agreement is the latest in a series of acquisi...,agreement is the latest in a series of and str...
9,"Jan 22, 2018",https://www.novartis.com/news/media-releases/n...,LIBERTY is the first migraine prevention trial...,Novartis reports erenumab met all primary and ...,NVS,liberty is the first migraine prevention trial...,liberty is the first migraine prevention trial...


In [16]:
#nonsense text seems to be gone
#df_4_companies_prep.at[1, 'article_text_clean']

In [17]:
#Pickle this dataframe and save as csv. It contains the raw article text, the article text w numbers 
#removed and all lowercase letters, and the text cleaned of nonsense words. I changed
#the filenames of the previous csv and pickle files by appending _old
df_4_companies_prep.to_csv('df_4_companies_prep.csv')
with open('df_4_companies_prep.pickle', 'wb') as to_write:
    pickle.dump(df_4_companies_prep, to_write)

In [18]:
#Let's drop a couple columns that we no longer need.
df_4_companies_prep.drop(columns=['article_text', 'article_text_lower'], inplace=True)

In [19]:
df_4_companies_prep.head(1)

,article_date,article_link-href,article_title,ticker,article_text_clean
0,"Apr 01, 2019",https://www.novartis.com/news/media-releases/n...,Novartis adds clinical and preclinical anti-in...,NVS,acquisition will give one clinical and two pre...


In [20]:
##NOT DOING THIS AGAIN
#Tokenize into trigrams 
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

In [21]:
#First, tokenize by words
df_4_companies_prep['article_text_words']=df_4_companies_prep['article_text_clean'].apply(word_tokenize)

In [22]:
df_4_companies_prep.head()

,article_date,article_link-href,article_title,ticker,article_text_clean,article_text_words
0,"Apr 01, 2019",https://www.novartis.com/news/media-releases/n...,Novartis adds clinical and preclinical anti-in...,NVS,acquisition will give one clinical and two pre...,"[acquisition, will, give, one, clinical, and, ..."
1,"Jun 15, 2017",https://www.novartis.com/news/media-releases/n...,Novartis' Cosentyx shows sustained improvement...,NVS,® is the first and only inhibitor to show sust...,"[®, is, the, first, and, only, inhibitor, to, ..."
2,"Jun 04, 2017",https://www.novartis.com/news/media-releases/n...,Novartis landmark study of Tafinlar® + Mekinis...,NVS,study is follow up to date of a and inhibitor ...,"[study, is, follow, up, to, date, of, a, and, ..."
3,"Apr 30, 2019",https://www.novartis.com/news/media-releases/s...,Sandoz enters agreement for proposed trastuzum...,NVS,collaboration in phase development for human e...,"[collaboration, in, phase, development, for, h..."
4,"Jun 23, 2017",https://www.novartis.com/news/media-releases/n...,Novartis pivotal CTL019 6-month follow-up data...,NVS,of complete remission or with incomplete blood...,"[of, complete, remission, or, with, incomplete..."


In [29]:
df_9_companies_prep.head(1)

,article_date,article_link-href,article_title,ticker,article_text_clean,article_text_words
0,"September 15, 2016 at 7:30 AM EDT",https://investors.biogen.com/news-releases/new...,New Data Presented at ECTRIMS Reinforce Effica...,BIIB,new analysis of “ no evidence of disease activ...,"[new, analysis, of, “, no, evidence, of, disea..."


In [30]:
###for 4 companies notebook, not doing this
###THIS WORKED BUT I SHOULD HAVE REMOVED STOP WORDS FIRST. SEE BELOW###
#This should make each word_tokenized story in the article_text_words column of the dataframe
#into a list of trigrams, and then put that list as a new column in the dataframe.

article_text_trigrams_list=[]
for article in df_9_companies_prep['article_text_words']:
    trigrams = list(ngrams(article, 3))
    article_text_trigrams_list.append(trigrams)
df_9_companies_prep['article_text_trigrams']=article_text_trigrams_list

In [31]:
df_9_companies_prep.head()

,article_date,article_link-href,article_title,ticker,article_text_clean,article_text_words,article_text_trigrams
0,"September 15, 2016 at 7:30 AM EDT",https://investors.biogen.com/news-releases/new...,New Data Presented at ECTRIMS Reinforce Effica...,BIIB,new analysis of “ no evidence of disease activ...,"[new, analysis, of, “, no, evidence, of, disea...","[(new, analysis, of), (analysis, of, “), (of, ..."
1,"July 5, 2016 at 8:00 AM EDT",https://investors.biogen.com/news-releases/new...,Biogen and AbbVie’s Once-Monthly ZINBRYTA™ (Da...,BIIB,significantly reduced multiple of disease acti...,"[significantly, reduced, multiple, of, disease...","[(significantly, reduced, multiple), (reduced,..."
2,"December 20, 2016 at 9:00 AM EST",https://investors.biogen.com/news-releases/new...,Biogen to Present at the 35th Annual J.P. Morg...,BIIB,mass business wire biogen today that it will p...,"[mass, business, wire, biogen, today, that, it...","[(mass, business, wire), (business, wire, biog..."
3,"August 9, 2016 at 9:00 AM EDT",https://investors.biogen.com/news-releases/new...,Biogen Announces Bioverativ as Name of New Hem...,BIIB,will focus on innovation for people living wit...,"[will, focus, on, innovation, for, people, liv...","[(will, focus, on), (focus, on, innovation), (..."
4,"December 21, 2016 at 8:00 AM EST",https://investors.biogen.com/news-releases/new...,Biogen to Report Fourth Quarter and Year End 2...,BIIB,mass business wire biogen today it will report...,"[mass, business, wire, biogen, today, it, will...","[(mass, business, wire), (business, wire, biog..."


In [32]:
#looks like that worked. Pickle this dataframe. Save as df_9_companies_prep_trigrams.pickle,
#to indicate the column of trigrams.
with open('df_9_companies_prep_trigrams.pickle', 'wb') as to_write:
    pickle.dump(df_9_companies_prep, to_write)

In [33]:
#What I should have done was to remove the stop words after word_tokenizer, but before
#making the trigrams. Let's drop the trigrams column and try this again.
df_9_companies_prep.drop(columns=['article_text_trigrams'], inplace=True)
df_9_companies_prep.head(1)

,article_date,article_link-href,article_title,ticker,article_text_clean,article_text_words
0,"September 15, 2016 at 7:30 AM EDT",https://investors.biogen.com/news-releases/new...,New Data Presented at ECTRIMS Reinforce Effica...,BIIB,new analysis of “ no evidence of disease activ...,"[new, analysis, of, “, no, evidence, of, disea..."


In [44]:
#add words ('press', 'release', 'back', 'view', 'printer', 'friendly', 'version') to stopwords list
#need to use .extend(), NOT .append() to add more than one word!!
stop=stopwords.words('english')
print(len(stop))
extras=['press', 'release', 'back', 'view', 'printer', 'friendly', 'version']
stop.extend(extras)
print(len(stop))

179
186


In [46]:
#print(stop)

In [47]:
df_9_companies_prep['article_text_nostop']=(df_9_companies_prep['article_text_words']).apply(lambda x: [item for item in x if item not in stop])

In [48]:
df_9_companies_prep.head()

,article_date,article_link-href,article_title,ticker,article_text_clean,article_text_words,article_text_nostop
0,"September 15, 2016 at 7:30 AM EDT",https://investors.biogen.com/news-releases/new...,New Data Presented at ECTRIMS Reinforce Effica...,BIIB,new analysis of “ no evidence of disease activ...,"[new, analysis, of, “, no, evidence, of, disea...","[new, analysis, “, evidence, disease, activity..."
1,"July 5, 2016 at 8:00 AM EDT",https://investors.biogen.com/news-releases/new...,Biogen and AbbVie’s Once-Monthly ZINBRYTA™ (Da...,BIIB,significantly reduced multiple of disease acti...,"[significantly, reduced, multiple, of, disease...","[significantly, reduced, multiple, disease, ac..."
2,"December 20, 2016 at 9:00 AM EST",https://investors.biogen.com/news-releases/new...,Biogen to Present at the 35th Annual J.P. Morg...,BIIB,mass business wire biogen today that it will p...,"[mass, business, wire, biogen, today, that, it...","[mass, business, wire, biogen, today, present,..."
3,"August 9, 2016 at 9:00 AM EDT",https://investors.biogen.com/news-releases/new...,Biogen Announces Bioverativ as Name of New Hem...,BIIB,will focus on innovation for people living wit...,"[will, focus, on, innovation, for, people, liv...","[focus, innovation, people, living, hemophilia..."
4,"December 21, 2016 at 8:00 AM EST",https://investors.biogen.com/news-releases/new...,Biogen to Report Fourth Quarter and Year End 2...,BIIB,mass business wire biogen today it will report...,"[mass, business, wire, biogen, today, it, will...","[mass, business, wire, biogen, today, report, ..."


In [49]:
###NO TRIGRAMS
#OK! That worked. Let's make trigrams, but first drop the article_text_clean column to save space
df_9_companies_prep.drop(columns=['article_text_clean'], inplace=True)
df_9_companies_prep.head(1)

,article_date,article_link-href,article_title,ticker,article_text_words,article_text_nostop
0,"September 15, 2016 at 7:30 AM EDT",https://investors.biogen.com/news-releases/new...,New Data Presented at ECTRIMS Reinforce Effica...,BIIB,"[new, analysis, of, “, no, evidence, of, disea...","[new, analysis, “, evidence, disease, activity..."


In [50]:
#pickle this dataframe
with open('df_9_companies_prep_nostop.pickle', 'wb') as to_write:
    pickle.dump(df_9_companies_prep, to_write)

In [51]:
#Now make the trigrams. Use the code from above, as it worked.
article_text_trigrams_list=[]
for article in df_9_companies_prep['article_text_nostop']:
    trigrams = list(ngrams(article, 3))
    article_text_trigrams_list.append(trigrams)
df_9_companies_prep['article_text_trigrams']=article_text_trigrams_list

In [52]:
df_9_companies_prep.head(1)

,article_date,article_link-href,article_title,ticker,article_text_words,article_text_nostop,article_text_trigrams
0,"September 15, 2016 at 7:30 AM EDT",https://investors.biogen.com/news-releases/new...,New Data Presented at ECTRIMS Reinforce Effica...,BIIB,"[new, analysis, of, “, no, evidence, of, disea...","[new, analysis, “, evidence, disease, activity...","[(new, analysis, “), (analysis, “, evidence), ..."


In [53]:
#pickle this dataframe. I put the old trigrams pickle file in the trash.
with open('df_9_companies_prep_trigrams.pickle', 'wb') as to_write:
    pickle.dump(df_9_companies_prep, to_write)

In [ ]:
#next step is to lemmatize

In [56]:
!pip install cython

In [72]:
!conda install -c conda-forge gensim

Solving environment: done

# All requested packages already installed.



In [76]:
###GENSIM DID NOT WORK
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.utils import lemmatize
#It's telling me I need to install the pattern library. When I try to do that via pip 
#from the command line, I get an error: ERROR: Could not find a version that satisfies the requirement mysqlclient (from pattern)
#ERROR: No matching distribution found for mysqlclient
#I'm reading on https://github.com/clips/pattern that pattern supports python 3.6, 
#since I'm running 3.8, maybe it won't work with 3.8. Also Word2Vec is not working.

In [23]:
from nltk.stem import WordNetLemmatizer
wnl=WordNetLemmatizer()

In [24]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/amysillman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [83]:
print(wnl.lemmatize('dogs'))

dog


In [87]:
#I'm not sure it will lemmatize on the trigrams. I guess they're individual tokens, so 
#it should be able to.

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

df_9_companies_prep['words_lemmatized']=(df_9_companies_prep['article_text_nostop']).apply(lemmatize_text)


In [93]:
df_9_companies_prep.head(10)

,article_date,article_link-href,article_title,ticker,article_text_words,article_text_nostop,article_text_trigrams,words_lemmatized
0,"September 15, 2016 at 7:30 AM EDT",https://investors.biogen.com/news-releases/new...,New Data Presented at ECTRIMS Reinforce Effica...,BIIB,"[new, analysis, of, “, no, evidence, of, disea...","[new, analysis, “, evidence, disease, activity...","[(new, analysis, “), (analysis, “, evidence), ...","[new, analysis, “, evidence, disease, activity..."
1,"July 5, 2016 at 8:00 AM EDT",https://investors.biogen.com/news-releases/new...,Biogen and AbbVie’s Once-Monthly ZINBRYTA™ (Da...,BIIB,"[significantly, reduced, multiple, of, disease...","[significantly, reduced, multiple, disease, ac...","[(significantly, reduced, multiple), (reduced,...","[significantly, reduced, multiple, disease, ac..."
2,"December 20, 2016 at 9:00 AM EST",https://investors.biogen.com/news-releases/new...,Biogen to Present at the 35th Annual J.P. Morg...,BIIB,"[mass, business, wire, biogen, today, that, it...","[mass, business, wire, biogen, today, present,...","[(mass, business, wire), (business, wire, biog...","[mass, business, wire, biogen, today, present,..."
3,"August 9, 2016 at 9:00 AM EDT",https://investors.biogen.com/news-releases/new...,Biogen Announces Bioverativ as Name of New Hem...,BIIB,"[will, focus, on, innovation, for, people, liv...","[focus, innovation, people, living, hemophilia...","[(focus, innovation, people), (innovation, peo...","[focus, innovation, people, living, hemophilia..."
4,"December 21, 2016 at 8:00 AM EST",https://investors.biogen.com/news-releases/new...,Biogen to Report Fourth Quarter and Year End 2...,BIIB,"[mass, business, wire, biogen, today, it, will...","[mass, business, wire, biogen, today, report, ...","[(mass, business, wire), (business, wire, biog...","[mass, business, wire, biogen, today, report, ..."
5,"April 29, 2016 at 7:39 AM EDT",https://investors.biogen.com/news-releases/new...,Biogen and AbbVie Receive Positive Opinion fro...,BIIB,"[positive, benefit, risk, profile, of, by, lar...","[positive, benefit, risk, profile, large, head...","[(positive, benefit, risk), (benefit, risk, pr...","[positive, benefit, risk, profile, large, head..."
6,"April 27, 2016 at 9:15 AM EDT",https://investors.biogen.com/news-releases/new...,Biogen Appoints Michael Ehlers Executive Vice ...,BIIB,"[will, lead, company, ’, s, to, create, mass, ...","[lead, company, ’, create, mass, business, wir...","[(lead, company, ’), (company, ’, create), (’,...","[lead, company, ’, create, mass, business, wir..."
7,"June 20, 2016 at 4:01 PM EDT",https://investors.biogen.com/news-releases/new...,Biogen to Report Second Quarter 2016 Financial...,BIIB,"[mass, business, wire, biogen, today, it, will...","[mass, business, wire, biogen, today, report, ...","[(mass, business, wire), (business, wire, biog...","[mass, business, wire, biogen, today, report, ..."
8,"May 13, 2016 at 2:47 AM EDT",https://investors.biogen.com/news-releases/new...,Alprolix® (rFIXFc) Approved in the EU for the ...,BIIB,"[first, fusion, therapy, for, b, in, the, eu, ...","[first, fusion, therapy, b, eu, provide, exten...","[(first, fusion, therapy), (fusion, therapy, b...","[first, fusion, therapy, b, eu, provide, exten..."
9,"April 29, 2016 at 11:00 AM EDT",https://investors.biogen.com/news-releases/new...,"Adam Koppel, Executive Vice President, Strateg...",BIIB,"[mass, –, biogen, today, that, m, d, d, execut...","[mass, –, biogen, today, executive, vice, pres...","[(mass, –, biogen), (–, biogen, today), (bioge...","[mass, –, biogen, today, executive, vice, pres..."


In [95]:
#df_9_companies_prep.at[9, 'words_lemmatized']
#I'm not sure it worked...

In [96]:
#next step is to vectorize. Do both tf-idf and cv. Might want to pickle dataframe 
#and then take off a couple columns.
with open('df_9_companies_prep_lemmatized.pickle', 'wb') as to_write:
    pickle.dump(df_9_companies_prep, to_write)

In [98]:
#Load pickled dataframe from vectorize-topic modeling notebook
import pickle
with open('df_9_companies_prep_lemmatized.pickle', 'rb') as read_file:
    df_9_companies_prep=pickle.load(read_file)

In [25]:
##OK RESTART HERE. WE'RE GOING TO TAKE OUT STOPWORDS
df_4_companies_prep.head(1)

,article_date,article_link-href,article_title,ticker,article_text_clean,article_text_words
0,"Apr 01, 2019",https://www.novartis.com/news/media-releases/n...,Novartis adds clinical and preclinical anti-in...,NVS,acquisition will give one clinical and two pre...,"[acquisition, will, give, one, clinical, and, ..."


In [101]:
###WE DIDN'T LEMMATIZE THIS TIME
#need to drop a couple columns
df_9_companies_prep.drop(columns=['article_text_words', 'article_text_trigrams'], inplace=True)
df_9_companies_prep.head(1)

,article_date,article_link-href,article_title,ticker,article_text_nostop,words_lemmatized
0,"September 15, 2016 at 7:30 AM EDT",https://investors.biogen.com/news-releases/new...,New Data Presented at ECTRIMS Reinforce Effica...,BIIB,"[new, analysis, “, evidence, disease, activity...","[new, analysis, “, evidence, disease, activity..."


In [26]:
#take out more stopwords. Company names, words from the forward-looking statements
#put into the extras list and add to the stopwords below.
extras=['catalyst', 'clovis', 'eli', 'lilly', 'rigel', 'sanofi', 'aventis', 'theravance', 'vertex', 'novartis', 'biogen',
'press', 'release', 'back', 'view', 'printer', 'friendly', 'version', 
'exchange',
'commission',
'sec',
'forward',
'looking',
'speak',
'biogen',
'obligation',
'whether',
'please',
'visit',
'biogen',
'follow',
'twitter',
'biogen',
'safe',
'harbor',
'forward',
'looking',
'potential',
'safety',
'forward',
'looking',
'may',
'potential',
'anticipate',
'believe',
'estimate',
'expect',
'intend',
'may',
'plan',
'meaning',
'place',
'undue',
'reliance',
'could',
'materially',
'reflected',
'uncertainty',
'success',
'commercialization',
'may',
'impacted',
'acceptance',
'community',
'competition',
'market',
'marketing',
'process',
'occurrence',
'safety',
'availability',
'reimbursement',
'biogen',
'failure',
'obtain',
'regulatory',
'failure',
'protect',
'intellectual',
'property',
'proprietary',
'product',
'liability',
'party',
'risk',
'section',
'biogen',
'annual',
'quarterly',
'report',
'biogen',
'exchange',
'commission',
'sec',
'forward',
'looking',
'speak',
'biogen',
'obligation',
'forward',
'looking',
'whether',
'new',
'information',
'future',
'otherwise',
'global',
'based',
'company',
'formed',
'following',
'separation',
'company',
'’',
'mission',
'use',
'unique',
'approach',
'innovation',
'develop',
'market',
'address',
'world',
'together',
'wholly',
'subsidiary',
'information',
'company',
'portfolio',
'please',
'visit',
'follow',
'twitter',
'page',
'forward',
'looking',
'news',
'may',
'forward',
'looking',
'private',
'litigation',
'reform',
'act',
'believe',
'expect',
'anticipate',
'forward',
'looking',
'subject',
'materially',
'include',
'limited',
'intellectual',
'property',
'competition',
'inherent',
'process',
'litigation',
'government',
'action',
'applicable',
'industry',
'information',
'economic',
'competitive',
'governmental',
'technological',
'may',
'affect',
'set',
'forth',
'item',
'risk',
'annual',
'report',
'form',
'exchange',
'commission',
'obligation',
'publicly',
'forward',
'looking',
'except',
'law',
'supplementary',
'appendix',
'thumbnail',
'preview',
'square',
'small',
'full',
'size',
'contact',
'medium',
'contact',
'law',
'public',
'griffin',
'investor',
'contact',
'biogen',
'mike',
'shea', '"', 'year', 'york', 'ability', 'commercially', 'filing', 'commercial']
 

In [116]:
#finding words in the forward looking statements to take out. Looked at:
#df_9_companies_prep.at[0, 'words_lemmatized']

In [27]:
stop=stopwords.words('english')
print(len(stop))
#extras=['press', 'release', 'back', 'view', 'printer', 'friendly', 'version']
#see above for additions to the stop words list
stop.extend(extras)
print(len(stop))

179
390


In [28]:
#apply the stop words to the column article_text_words
df_4_companies_prep['article_words_nostop']=(df_4_companies_prep['article_text_words']).apply(lambda x: [item for item in x if item not in stop])

In [29]:
df_4_companies_prep.head()


,article_date,article_link-href,article_title,ticker,article_text_clean,article_text_words,article_words_nostop
0,"Apr 01, 2019",https://www.novartis.com/news/media-releases/n...,Novartis adds clinical and preclinical anti-in...,NVS,acquisition will give one clinical and two pre...,"[acquisition, will, give, one, clinical, and, ...","[acquisition, give, one, clinical, two, precli..."
1,"Jun 15, 2017",https://www.novartis.com/news/media-releases/n...,Novartis' Cosentyx shows sustained improvement...,NVS,® is the first and only inhibitor to show sust...,"[®, is, the, first, and, only, inhibitor, to, ...","[®, first, inhibitor, show, sustained, spondyl..."
2,"Jun 04, 2017",https://www.novartis.com/news/media-releases/n...,Novartis landmark study of Tafinlar® + Mekinis...,NVS,study is follow up to date of a and inhibitor ...,"[study, is, follow, up, to, date, of, a, and, ...","[study, date, inhibitor, combination, therapy,..."
3,"Apr 30, 2019",https://www.novartis.com/news/media-releases/s...,Sandoz enters agreement for proposed trastuzum...,NVS,collaboration in phase development for human e...,"[collaboration, in, phase, development, for, h...","[collaboration, phase, development, human, epi..."
4,"Jun 23, 2017",https://www.novartis.com/news/media-releases/n...,Novartis pivotal CTL019 6-month follow-up data...,NVS,of complete remission or with incomplete blood...,"[of, complete, remission, or, with, incomplete...","[complete, remission, incomplete, blood, count..."


In [30]:
#pickle this dataframe and use it in the vectorize-topic modeling next notebook
with open('df_4_companies_prep_nostop.pickle', 'wb') as to_write:
    pickle.dump(df_4_companies_prep, to_write)

In [ ]:
#take out quotation marks
df_9_companies_prep['article_text_lower_noq'] = df_9_companies_prep['article_text_lower'].str.strip(to_strip='"')

In [ ]:
#force into a string data type
katniss['story_text']=katniss['story_text'].astype(str)

In [13]:
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

sent = "Io andiamo to the beach with my amico."
" ".join(w for w in nltk.wordpunct_tokenize(sent) \
     if w.lower() in words or not w.isalpha())

[nltk_data] Downloading package words to
[nltk_data]     /Users/amysillman/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


'Io to the beach with my .'